In [ ]:
# 1、了解数据
## 1）加载必要的库
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## 2） 读取数据
user=pd.read_csv('/home/kesci/tianchi_fresh_comp_train_user.csv')

## 3）快速查看数据类型和数据结构
user.info()

# 快速查看统计信息
user.describe()

## 4）统计缺失值
user.isnull().sum()

##暂时不做地理数据的分析，user_geohash 这列的缺失值暂不做处理。



# 2、处理数据
## 1）删除重复值
user.drop_duplicates(keep='last',inplace=True)

## 2）将time转换为datetime格式
user['time']=pd.to_datetime(user['time'])

## 3）提取出日期和时间
user['dates'] = user.time.dt.date
user['month'] = user.dates.values.astype('datetime64[M]')
user['hours'] = user.time.dt.hour

## 4）转换数据类型
user['behavior_type']=user['behavior_type'].apply(str)
user['user_id']=user['user_id'].apply(str)
user['item_id']=user['item_id'].apply(str)

#  3、数据分析——可视化
## 1） 统计每日PV和UV数据
pv_day=user[user.behavior_type=="1"].groupby("dates")["behavior_type"].count()
uv_day=user[user.behavior_type=="1"].drop_duplicates(["user_id","dates"]).groupby("dates")["user_id"].count()

## 2）分析每天的pv与uv的趋势
# 加载库
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.charts import Grid
import numpy as np

# 做出每天的pv与uv趋势图
attr=list(pv_day.index)
pv=(
    Line(init_opts=opts.InitOpts(width="1000px",height="500px"))
    .add_xaxis(xaxis_data=attr)
    .add_yaxis(
        "pv",
        np.around(pv_day.values/10000,decimals=2),
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        series_name="uv",
        yaxis_index=1,
        y_axis=np.around(uv_day.values/10000,decimals=2),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="uv",
            type_="value",
            min_=0,
            max_=1.6,
            interval=0.4,
            axislabel_opts=opts.LabelOpts(formatter="{value} 万人"),
        )
    )
    .set_global_opts(
        tooltip_opts=opts.TooltipOpts(
            is_show=True,trigger="axis",axis_pointer_type="cross"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow"),
        ),
        yaxis_opts=opts.AxisOpts(
            name="pv",
            type_="value",
            min_=0,
            max_=100,
            interval=20,
            axislabel_opts=opts.LabelOpts(formatter="{value} 万次"),
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        title_opts=opts.TitleOpts(title="pv与uv趋势图"),
    )
)

pv.render_notebook()

## 3） pv、uv差异分析（by day）
pv_uv = pd.concat([pv_day, uv_day], join='outer', axis=1)
pv_uv.columns = ['pv_day', 'uv_day']

new_day=pv_uv.diff()
new_day.columns=['new_pv','new_uv']
new_day

attr = new_day.index
v = new_day.new_uv
w = new_day.new_pv

li=(
    Line(init_opts=opts.InitOpts(width="1000px",height="500px"))
    .add_xaxis(xaxis_data=attr)
    .add_yaxis(
        "新增pv",
        w,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="新增uv",
            type_="value",
            min_=-2000,
            max_=1600,
            interval=400,
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
        )
    )
     .set_global_opts(
        tooltip_opts=opts.TooltipOpts(
            is_show=True, trigger="axis", axis_pointer_type="cross"
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axispointer_opts=opts.AxisPointerOpts(is_show=True, type_="shadow"),
        ),
        yaxis_opts=opts.AxisOpts(
            name="新增pv",
            type_="value",
            min_=-350000,
            max_=250000,
            interval=100000,
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
         title_opts=opts.TitleOpts(title="pv、uv差异分析"),
    )
)
il=(
    Line()
    .add_xaxis(xaxis_data=attr)
    .add_yaxis("新增uv",v,yaxis_index='1',label_opts=opts.LabelOpts(is_show=False),)
)
c=li.overlap(il)
c.render_notebook()


## 4） 不同时期用户行为分析
shopping_cart= user[user.behavior_type == '3'].groupby('dates')['behavior_type'].count()
collect=user[user.behavior_type=='2'].groupby('dates')['behavior_type'].count()
buy=user[user.behavior_type=='4'].groupby('dates')['behavior_type'].count()

attr_a=list(shopping_cart.index)
v_1=shopping_cart.values.tolist()
v_2=collect.values.tolist()
v_3=buy.values.tolist()

b=(
    Line()
    .add_xaxis(xaxis_data=attr_a)
    .add_yaxis(
        "加购人数",
        v_1,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        "收藏人数",
        v_2,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        "购买人数",
        v_3,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="不同时期用户行为数据"))
)
b.render_notebook()



#  4、把数据拆分为活动数据和日常数据做不同时段的分析
#####存在双12大促的数据，用户的不同时段行为可能会影响分析结果，因此拆分时段，做对比分析。

## 1）把dates列转换为datetime类型
user['dates']=pd.to_datetime(user['dates'])

## 2）拆分数据子集（活动时段、日常时段）
active=user[user["dates"].isin(["2014/12/11","2014/12/12","2014/12/13"])]
daily=user[~user["dates"].isin(["2014/12/11","2014/12/12","2014/12/13"])]

## 3）活动期间不同时段的用户行为分析
from pyecharts.charts import Bar
# 活动数据
cart_h= active[active.behavior_type == '3'].groupby('hours')['behavior_type'].count()
collect_h=active[active.behavior_type=='2'].groupby('hours')['behavior_type'].count()
buy_h=active[active.behavior_type=='4'].groupby('hours')['behavior_type'].count()
uv_h=active[active.behavior_type== '1'].groupby('hours')['user_id'].count()

attr_h=list(cart_h.index)
h1=np.around(cart_h.values/3,decimals=0).tolist()
h2=np.around(collect_h.values/3,decimals=0).tolist()
h3=np.around(buy_h.values/3,decimals=0).tolist()
h4=np.around(uv_h.values/3,decimals=0).tolist()


h=(
    Line(init_opts=opts.InitOpts(width="1000px",height="500px"))
    .add_xaxis(xaxis_data=attr_h)
    .add_yaxis(
        "加购人数",
        h1,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        "收藏人数",
        h2,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        "购买人数",
        h3,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=15)),
        title_opts=opts.TitleOpts(title="日均各时段活动用户行为",pos_top="48%"),
        legend_opts=opts.LegendOpts(pos_top="48%"),
    )
)
bar=(
    Bar()
    .add_xaxis(xaxis_data=attr_h)
    .add_yaxis(
    "浏览人数",
        h4,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="活动pv对比数据"),
    )
)

ggrid = (
    Grid()
    .add(bar, grid_opts=opts.GridOpts(pos_bottom="60%"))
    .add(h, grid_opts=opts.GridOpts(pos_top="60%"))
)
ggrid.render_notebook()


## 上图结论：这是三天活动的日均数据，可以发现活动期间是商家在起主导作用大促集中在零点，因此用户的购买高峰也出现在0点，点击浏览的高峰集中在晚上的21点到22点之间，因此商家可以在20点前改好促销页面吸引顾客参加0点的活动。


## 4）日常期间不同时段的用户行为分析
# 日常数据
cart_d= daily[daily.behavior_type == '3'].groupby('hours')['behavior_type'].count()
collect_d=daily[daily.behavior_type=='2'].groupby('hours')['behavior_type'].count()
buy_d=daily[daily.behavior_type=='4'].groupby('hours')['behavior_type'].count()
uv_d=daily[daily.behavior_type== '1'].groupby('hours')['user_id'].count()

attr_d=list(cart_d.index)
d1=np.around(cart_d.values/28,decimals=0).tolist()
d2=np.around(collect_d.values/28,decimals=0).tolist()
d3=np.around(buy_d.values/28,decimals=0).tolist()
d4=np.around(uv_d.values/3,decimals=0).tolist()

d=(
    Line(init_opts=opts.InitOpts(width="1000px",height="500px"))
    .add_xaxis(xaxis_data=attr_d)
    .add_yaxis(
        "加购人数",
        d1,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        "收藏人数",
        d2,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        "购买人数",
        d3,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=15)),
        title_opts=opts.TitleOpts(title="日均各时段活动用户行为",pos_top="48%"),
        legend_opts=opts.LegendOpts(pos_top="48%"),
    )
)
y=(
    Bar()
    .add_xaxis(xaxis_data=attr_d)
    .add_yaxis(
    "浏览人数",
        d4,
        label_opts=opts.LabelOpts(is_show=False)
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="日常pv对比数据"),
    )
)

ggrid = (
    Grid()
    .add(y, grid_opts=opts.GridOpts(pos_bottom="60%"))
    .add(d, grid_opts=opts.GridOpts(pos_top="60%"))
)
ggrid.render_notebook()


## 上图结论：与大促不同的是日常期间购买人数从上午10点到晚上23点变化都不会太大高峰出现在晚上21点，pv、加购、收藏的高峰出现在晚上21点到22点之间，说明大家都喜欢在晚上这个时间段浏览商品，日常时段可以集中在这个时段进行促销活动，浏览高峰也是集中在晚上21点到22点之间。

# 活动时购买率
hour_buy_user_num = active[active.behavior_type == '4'].drop_duplicates(['user_id','dates', 'hours']).groupby('hours')['user_id'].count()
hour_active_user_num = active.drop_duplicates(['user_id','dates', 'hours']).groupby('hours')['user_id'].count()
hour_buy_rate = hour_buy_user_num / hour_active_user_num
attr_o = list(hour_buy_user_num.index)
vo_2 =np.around(hour_buy_rate.values,decimals=2) 
# 日常时购买率
hour_buy_daily_num = daily[daily.behavior_type == '4'].drop_duplicates(['user_id','dates', 'hours']).groupby('hours')['user_id'].count()
hour_active_daily_num = daily.drop_duplicates(['user_id','dates', 'hours']).groupby('hours')['user_id'].count()
daily_buy_rate = hour_buy_daily_num / hour_active_daily_num
vi_2 =np.around(daily_buy_rate.values,decimals=2) 


hbu=(
    Line()
    .add_xaxis(xaxis_data=attr_o)
    .add_yaxis(
        "日常购买率",
        vi_2,
    )
    .add_yaxis(
        "活动购买率",
        vo_2,
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="不同时段购买率"))
)

hbu.render_notebook()

## 上图结论：日常时的购买率最高的出现在上午10点到下午15点间，还有晚上的21点，和活动期间的购买率不同，但是明显晚上21点已经在分析中出现比较多的峰值，因此可以考虑这个时段做做吸引用户购买的措施。


## 5）转化漏斗分析
### 1）活动时段的转化漏斗
from pyecharts.charts import Funnel

# 活动转化
a_pv=active[active.behavior_type=="1"]["user_id"].count()
a_cart=active[active.behavior_type=="3"]["user_id"].count()
a_collect=active[active.behavior_type=="2"]["user_id"].count()
a_buy=active[active.behavior_type=="4"]["user_id"].count()

a_attr=["点击","加入购物车","收藏","购买"]
values=[np.around((a_pv/a_pv*100),2),
        np.around((a_cart/a_pv*100),2),
        np.around((a_collect/a_pv*100),2),
        np.around((a_buy/a_pv*100),2),
       ]

data = [[a_attr[i], values[i]] for i in range(len(a_attr))]

a=(
    Funnel()
    .add(
        series_name="用户行为",
        data_pair=data,
        gap=2,
        tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{a} <br/>{b} : {c}%",is_show=True),
        label_opts=opts.LabelOpts(is_show=True, position="ourside"),
        itemstyle_opts=opts.ItemStyleOpts(border_color="#fff", border_width=1),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="用户转化漏斗", subtitle="活动"))
)

a.render_notebook()

## 上图结论：活动期间日均从点击到加入购物车的转化率只有4.97%，购买的只有2%，说明点击浏览量不少但是吸引不了顾客购买，虽然是大的活动，但是转化率还是很低的，可以从提高加购率和收藏率着手，从而吸引顾客购买。


### 2）日常时段的转化漏斗
# 日常转化
l_pv=daily[daily.behavior_type=="1"]["user_id"].count()
l_cart=daily[daily.behavior_type=="3"]["user_id"].count()
l_collect=daily[daily.behavior_type=="2"]["user_id"].count()
l_buy=daily[daily.behavior_type=="4"]["user_id"].count()

l_attr=["点击","加入购物车","收藏","购买"]
valuel=[np.around((l_pv/l_pv*100),2),
        np.around((l_cart/l_pv*100),2),
        np.around((l_collect/l_pv*100),2),
        np.around((l_buy/l_pv*100),2),
       ]

datal = [[l_attr[i], valuel[i]] for i in range(len(l_attr))]

dy=(
    Funnel()
    .add(
        series_name="用户行为",
        data_pair=datal,
        gap=2,
        tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{a} <br/>{b} : {c}%",is_show=True),
        label_opts=opts.LabelOpts(is_show=True, position="ourside"),
        itemstyle_opts=opts.ItemStyleOpts(border_color="#fff", border_width=1),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="用户转化漏斗", subtitle="日常"))
)
dy.render_notebook()


## 上图结论：——日常期间总的点击量中，有4.45%加入购物车，有3.3%收藏，而到最后只有1.4%购买，整体来看，购买的转化率最低，有很大的增长空间；
#           就颜色来看，红色部分的变化最大，即“点击-加入购物车“这一环节的转化率最低，按照“点击-加入购物车-收藏-购买”这一用户行为路径，我们可通过优化“点击-加入购物车”这一环节进而提升购买的转化率，可以通过鼓励用户收藏加购后可以领券来刺激用户加购收藏从而刺激用户的购买欲望。